## Hotel reviews 1


In [2]:
import pandas as pd
import time

print('Loading data file now, this could take a while depending on file size')
start = time.time()

df = pd.read_csv("Hotel_Reviews.csv")

end = time.time()

print("Loading took " + str(round(end - start, 2)) + " seconds")

Loading data file now, this could take a while depending on file size
Loading took 1.13 seconds


In this case the data is already clean

## Dataframe operations

In [3]:
print("The shape of the data (rows, cols) is", str(df.shape))

The shape of the data (rows, cols) is (515738, 17)


- How many distinct values are there for the column Reviewer_Nationality and what are they?
- What reviewer nationality is the most common in the dataset (print country and number of reviews)?

In [5]:
nationality_freq = df['Reviewer_Nationality'].value_counts()
print("There are " + str(nationality_freq.size) + " different nationalities\n")
print(nationality_freq)

There are 227 different nationalities

Reviewer_Nationality
United Kingdom               245246
United States of America      35437
Australia                     21686
Ireland                       14827
United Arab Emirates          10235
                              ...  
Cape Verde                        1
Northern Mariana Islands          1
Tuvalu                            1
Guinea                            1
Palau                             1
Name: count, Length: 227, dtype: int64


What are the next top 10 most frequently found nationalities, and their frequency count?

In [10]:
print("The highest frequency reviewer nationality is " + str(nationality_freq.index[0]).strip() + " with " + str(nationality_freq.iloc[0]) + " reviews.")
# Notice there is a leading space on the values, strip() removes that for printing

print("The next 10 highest frequency reviewer nationalities are: ")
print(nationality_freq[1:11].to_string())

The highest frequency reviewer nationality is United Kingdom with 245246 reviews.
The next 10 highest frequency reviewer nationalities are: 
Reviewer_Nationality
United States of America     35437
Australia                    21686
Ireland                      14827
United Arab Emirates         10235
Saudi Arabia                  8951
Netherlands                   8772
Switzerland                   8678
Germany                       7941
Canada                        7894
France                        7296


What was the most frequently reviewed hotel for each of the top 10 most reviewer nationalities?

In [11]:
# Normally with pandas you will avoid an explicit loop, but wanted to show creating a new dataframe using criteria
# dont do this with large amounts of data because it could be very slow
for nat in nationality_freq[:10].index:
    # First, extract all the rows that match the criteria into a new dataframe
    nat_df = df[df['Reviewer_Nationality'] == nat]
    # Now get the hotel freq
    freq = nat_df["Hotel_Name"].value_counts()
    print("The most reviewed hotel for " + str(nat).strip() + " was " + str(freq.index[0]) + " with " + str(freq.iloc[0]) + " reviews.") 


The most reviewed hotel for United Kingdom was Britannia International Hotel Canary Wharf with 3833 reviews.
The most reviewed hotel for United States of America was Hotel Esther a with 423 reviews.
The most reviewed hotel for Australia was Park Plaza Westminster Bridge London with 167 reviews.
The most reviewed hotel for Ireland was Copthorne Tara Hotel London Kensington with 239 reviews.
The most reviewed hotel for United Arab Emirates was Millennium Hotel London Knightsbridge with 129 reviews.
The most reviewed hotel for Saudi Arabia was The Cumberland A Guoman Hotel with 142 reviews.
The most reviewed hotel for Netherlands was Jaz Amsterdam with 97 reviews.
The most reviewed hotel for Switzerland was Hotel Da Vinci with 97 reviews.
The most reviewed hotel for Germany was Hotel Da Vinci with 86 reviews.
The most reviewed hotel for Canada was St James Court A Taj Hotel London with 61 reviews.


How many reviews are there per hotel (frequency count of hotel) in the dataset?

In [13]:
# First create a new dataframe based on the old one, removing the uneeded columns
hotel_freq_df = df.drop(["Hotel_Address", "Additional_Number_of_Scoring", "Review_Date", "Average_Score", "Reviewer_Nationality", "Negative_Review", "Review_Total_Negative_Word_Counts", "Positive_Review", "Review_Total_Positive_Word_Counts", "Total_Number_of_Reviews_Reviewer_Has_Given", "Reviewer_Score", "Tags", "days_since_review", "lat", "lng"], axis = 1)

# Group the rows by Hotel_Name, count them and put the result in a new column Total_Reviews_Found
hotel_freq_df['Hotel_Reviews_Found'] = hotel_freq_df.groupby('Hotel_Name').transform('count')

# Get rid of all the duplicated rows
hotel_freq_df = hotel_freq_df.drop_duplicates(subset=["Hotel_Name"])
display(hotel_freq_df)

,Hotel_Name,Total_Number_of_Reviews,Hotel_Reviews_Found
0,Hotel Arena,1403,405
405,K K Hotel George,1831,566
971,Apex Temple Court Hotel,2619,1037
2008,The Park Grand London Paddington,4380,1770
3778,Monhotel Lounge SPA,171,35
...,...,...,...
511962,Suite Hotel 900 m zur Oper,3461,439
512401,Hotel Amadeus,717,144
512545,The Berkeley,232,100
512645,Holiday Inn London Kensington,5945,2768


The counted in the dataset results do not match the value in `Total_Number_of_Reviews`. It is unclear if this value in the dataset represented the total number of reviews the hotel had, but not all were scraped, or some other calculation. `Total_Numbe_of_Reviews` is not used in the model bacause of this unclarity

While there is a `Average_Score` column for each hotel in the dataset, you can also calculate an average score (getting the average of all reviewer scores in the dataset for each hotel). Add a new column to your dataframe with the column header `Calc_Average_Score` that contains that calculated average. Print out the columns `Hotel_Name`, `Average_Score`, and `Calc_Average_Score`.

In [15]:
# define a function that takes a row and performs some calculation with it
def get_difference_review_avg(row):
    return row['Average_Score'] - row['Calc_Average_Score']

# 'mean' is mathematical word for 'average'
df['Calc_Average_Score'] = round(df.groupby('Hotel_Name').Reviewer_Score.transform('mean'), 1)

# Add a new column with the difference between the two average scores
df['Average_Score_Difference'] = df.apply(get_difference_review_avg, axis=1)

# Create a df without all the duplicated of Hotel_Name (so only 1 row per hotel)
review_scores_df = df.drop_duplicates(subset = ['Hotel_Name'])

# Sort the dataframe to find the lowest and highest average score difference
review_scores_df = review_scores_df.sort_values(by=['Average_Score_Difference'])

display(review_scores_df[["Average_Score_Difference", "Average_Score", "Calc_Average_Score", "Hotel_Name"]])

,Average_Score_Difference,Average_Score,Calc_Average_Score,Hotel_Name
495945,-0.8,7.7,8.5,Best Western Hotel Astoria
111027,-0.7,8.8,9.5,Hotel Stendhal Place Vend me Paris MGallery by...
43688,-0.7,7.5,8.2,Mercure Paris Porte d Orleans
178253,-0.7,7.9,8.6,Renaissance Paris Vendome Hotel
218258,-0.5,7.0,7.5,Hotel Royal Elys es
...,...,...,...,...
151416,0.7,7.8,7.1,Best Western Allegro Nation
22189,0.8,7.1,6.3,Holiday Inn Paris Montparnasse Pasteur
250308,0.9,8.6,7.7,MARQUIS Faubourg St Honor Relais Ch teaux
68936,0.9,6.8,5.9,Villa Eugenie


You may also wonder about the Average_Score value and why it is sometimes different from the calculated average score. As we can't know why some of the values match, but others have a difference, it's safest in this case to use the review scores that we have to calculate the average ourselves.

With only 1 hotel having a difference of score greater than 1, it means we can probably ignore the difference and use the calculated average score.

- Calculate and print out how many rows have column Negative_Review values of "No Negative"
- Calculate and print out how many rows have column Positive_Review values of "No Positive"
- Calculate and print out how many rows have column Positive_Review values of "No Positive" and Negative_Review values of "No Negative"

In [16]:
# with lambdas 
start = time.time()

no_negative_reviews = df.apply(lambda x: True if x['Negative_Review'] == 'No Negative' else False, axis=1)
print('Number of No Negative reviews: ' + str(len(no_negative_reviews[no_negative_reviews == True].index)))

no_positive_reviews = df.apply(lambda x: True if x['Positive_Review'] == 'No Positive' else False, axis=1)
print('Number of No Positive reviews: ' + str(len(no_positive_reviews[no_positive_reviews == True].index)))

both_no_reviews = df.apply(lambda x: True if x['Negative_Review'] == "No Negative" and x['Positive_Review'] == "No Positive" else False , axis=1)
print("Number of both No Negative and No Positive reviews: " + str(len(both_no_reviews[both_no_reviews == True].index)))

end = time.time()

print('Lambdas took ' + str(round(end - start, 2)) + " seconds")

Number of No Negative reviews: 127890
Number of No Positive reviews: 35946
Number of both No Negative and No Positive reviews: 127
Lambdas took 2.68 seconds


## Another way

In [17]:
# without lambdas (using a mixture of notations to show you can use both)
start = time.time()
no_negative_reviews = sum(df.Negative_Review == "No Negative")
print("Number of No Negative reviews: " + str(no_negative_reviews))

no_positive_reviews = sum(df["Positive_Review"] == "No Positive")
print("Number of No Positive reviews: " + str(no_positive_reviews))

both_no_reviews = sum((df.Negative_Review == "No Negative") & (df.Positive_Review == "No Positive"))
print("Number of both No Negative and No Positive reviews: " + str(both_no_reviews))

end = time.time()
print("Sum took " + str(round(end - start, 2)) + " seconds")

Number of No Negative reviews: 127890
Number of No Positive reviews: 35946
Number of both No Negative and No Positive reviews: 127
Sum took 0.11 seconds


You may have noticed that there are 127 rows that have both "No Negative" and "No Positive" values for the columns Negative_Review and Positive_Review respectively. That means that the reviewer gave the hotel a numerical score, but declined to write either a positive or negative review. Luckily this is a small amount of rows (127 out of 515738, or 0.02%), so it probably won't skew our model or results in any particular direction, but you might not have expected a data set of reviews to have rows with no reviews, so it's worth exploring the data to discover rows like this.